# Atelier sur la génétique statistique et l'épidémiologie génétique STAGE-Québec
## Thème 2 - Phénotypes moléculaires en épidémiologie génétique

Par Marc-André Legault (Université de Montréal) et Qihuang Zhang (Université McGill)

**31 juillet - 1er août 2025**

### Introduction

Dans cet atelier, nous examinerons l'analyse d'association à l'échelle du transcriptome (TWAS) pour identifier les gènes associés au diabète de type 2 (DT2). L'objectif est de tester l'association entre l'expression génique prédite dans divers tissus et le développement du DT2.

Pour cette analyse, nous utiliserons des statistiques d'association sommaires provenant d'une méta-analyse récente de grande envergure impliquant 180 834 cas avec DT2 et plus d'un million de témoins [Mahajan _et al._ (2022)](https://www.nature.com/articles/s41588-022-01058-3). Nous mettrons en œuvre deux approches méthodologiques complémentaires : [S-PrediXcan](https://www.nature.com/articles/s41467-018-03621-1) et [TWAS FUSION](https://www.nature.com/articles/ng.3506) pour effectuer nos analyses TWAS.

**Préparation des données :** Les statistiques sommaires GWAS ont été pré-traitées pour s'assurer qu'elles sont alignées sur la même version du génome (GRCh38) que les modèles de prédiction d'expression génique distribués par ces logiciels. Pour optimiser la charge computationnelle de cet atelier, nous nous concentrerons sur un sous-ensemble représentatif de tissus de GTEx.

### Structure de l'atelier

Le contenu de l'atelier est organisé en trois sections progressives :

1. **S-PrediXcan** (ce carnet) - Exploration et mise en pratique
2. **TWAS / FUSION** [(lien)](2a-FUSION.ipynb) - Approche alternative et complémentaire  
3. **Comparaison des deux approches** [(lien)](3a-compare_TWAS_models.ipynb) - Analyse comparative des résultats

**Navigation :** Utilisez le navigateur de fichiers dans le panneau de gauche pour parcourir le contenu de l'atelier.

<div class="alert alert-warning">
    <p>
    <strong>Important pour la sauvegarde de vos travaux</strong>
    </p>
    <p>
    Le dossier <strong>local</strong> est partagé avec votre ordinateur et vos modifications y seront conservées. Tout le reste ne sera pas sauvegardé si vous fermez le serveur de carnets. Assurez-vous que tout ce qui devrait persister est stocké dans le dossier local.
    </p>
    <p>
    Le contenu de ce dossier est stocké dans votre répertoire personnel :<br />
    • Windows : <code>C:\Users\[votre nom d'utilisateur]\STAGE2025_workshop_theme2</code><br />
    • macOS/Linux : <code>/Users/[votre nom d'utilisateur]/STAGE2025_workshop_theme2</code>
    </p>
</div>

Nous pouvons maintenant commencer notre exploration en examinant S-PrediXcan et en l'appliquant avec nos données GWAS pré-traitées.

### Exploration des statistiques sommaires GWAS

Avant de procéder à l'analyse TWAS, il convient d'examiner nos données d'entrée. Les statistiques sommaires GWAS ont été harmonisées avec les modèles [PredictDB](https://predictdb.hakyimlab.org/), qui constituent les modèles de prédiction d'expression génique distribués comme données complémentaires à S-PrediXcan. 

**Structure des données :** Le format des données est standardisé et identique pour chaque tissu analysé. Examinons la structure de ces fichiers :

In [ ]:
filename="../data/PrediXcan/gwas_harmonized/harmonized_en_Whole_Blood.tsv.gz"
echo "Looking at file $filename"
echo "Column numbers and titles:"
echo "$(zcat $filename | head -n 1 | tr '\t' '\n' | nl)"
echo
echo "File excerpt:"
zcat $filename | head -n 10 | cut -f 1-6,9

<div class="alert alert-info">
    <strong>Question de réflexion</strong><br />
    Examinez attentivement les colonnes affichées ci-dessus. Que représente chacune d'elles dans le contexte d'une analyse GWAS ? Pourquoi ces informations sont-elles essentielles pour l'analyse TWAS ?
</div>

### Modèles de prédiction d'expression génique

Maintenant que nous avons exploré nos données GWAS, examinons les modèles de prédiction d'expression génique. Ces modèles constituent le cœur de l'approche S-PrediXcan : ils permettent de prédire l'expression des gènes à partir des génotypes des individus.

**Examen du tissu « Sang total » de GTEx :** Nous allons examiner en détail les modèles pour ce tissu, qui est couramment utilisé en épidémiologie génétique. Dans PredictDB, les modèles sont stockés sous forme de bases de données sqlite3 comprenant deux tables principales : `extra` et `weights`. Explorons leur structure :

In [ ]:
# Schéma pour la table "extra" :
sqlite3 ../data/PrediXcan/predictdb/en_Whole_Blood.db ".schema extra"

**Table `extra` - Métadonnées des modèles :** Cette table contient les métadonnées cruciales sur le modèle prédictif pour chaque gène. Vous y trouverez :
- Le nom du gène et son ID Ensembl
- Le nombre de SNP inclus dans le modèle  
- Les statistiques de performance prédictive du modèle d'expression génique

<div class="alert alert-info">
    <strong>Point clé à retenir</strong><br />
    Réfléchissez à l'importance de la colonne <code>pred.perf.pval</code>. À quoi sert cette valeur p dans le contexte de la prédiction d'expression génique ? Pourquoi est-il crucial de filtrer les gènes basés sur cette statistique avant de procéder à l'analyse TWAS ?
</div>

**Table `weights` - Architecture des modèles prédictifs :** Cette table contient l'information détaillée sur chaque modèle d'expression génique. Pour chaque `gene` (ID Ensembl du gène), elle liste les variants utilisés pour prédire l'expression génique (c.-à-d. les eQTL) accompagnés de leur `weight` de prédiction respectif.

**Exemple concret :** Examinons le modèle utilisé pour prédire l'expression du gène ENSG00000157823.16 (Adaptor Related Protein Complex 3 Subunit Sigma 2, _AP3S2_). Pour faciliter l'interprétation, nous trierons les variants par poids absolu et n'afficherons que les 10 principaux contributeurs à la prédiction :

In [ ]:
sqlite3 ../data/PrediXcan/predictdb/en_Whole_Blood.db <<EOF
.mode columns
.header on
select * from weights
  where gene='ENSG00000157823.16'
  order by abs(weight) desc limit 10;
EOF

<div class="alert alert-info">
    <strong>Questions d'approfondissement</strong><br />
    1. <strong>Interprétation des poids :</strong> Pourquoi observons-nous des poids à la fois positifs et négatifs ? Que nous indiquent ces signes sur la relation entre le variant et l'expression génique ?<br /><br />
    2. <strong>Méthodes d'estimation :</strong> Pouvez-vous mentionner une approche statistique couramment utilisée pour estimer ces poids de prédiction ? Indice : pensez aux méthodes de régression pénalisée.
</div>

### Analyse TWAS utilisant S-PrediXcan

Maintenant que nous avons exploré en détail la structure des modèles de prédiction d'expression génique PredictDB et que nous disposons de nos statistiques sommaires GWAS harmonisées, nous pouvons procéder à notre analyse TWAS avec S-PrediXcan.

**Temps d'exécution :** Le code ci-dessous peut prendre quelques minutes à s'exécuter selon les ressources disponibles. Si l'exécution prend trop de temps, vous pouvez l'interrompre en utilisant le bouton d'arrêt en haut de cette fenêtre et utiliser les fichiers de points de contrôle disponibles dans le dépôt.

#### Configuration des paramètres d'entrée

L'appel à SPrediXcan nécessite trois fichiers d'entrée essentiels :

- **`model_db_path`** : Contient les poids pour prédire l'expression des gènes inclus dans l'analyse TWAS. Il s'agit de la base de données sqlite3 fournie par PredictDB que nous venons d'explorer.

- **`covariance`** : Ce fichier contient les estimations de déséquilibre de liaison (DL) entre les SNP. Il permet de tenir compte de la structure de covariance entre les variants génétiques, un élément crucial pour obtenir des statistiques d'association correctes.

- **`gwas_file`** : Le fichier de statistiques sommaires GWAS contenant les associations entre les variants génétiques et le DT2. Ces résultats GWAS ont été préalablement harmonisés pour assurer la cohérence avec les variants disponibles dans PredictDB.

#### Paramètres de formatage des données

Les paramètres avec le suffixe `_column` indiquent au logiciel comment interpréter le fichier de statistiques sommaires GWAS. Vous pouvez vérifier que les noms fournis ci-dessous correspondent bien aux colonnes que nous avons examinées dans l'extrait du fichier de statistiques sommaires GWAS précédent.

#### Correction pour la polygénicité

Finalement, nous fournissons une estimation de l'héritabilité SNP ($h^2$) pour notre phénotype et la taille d'échantillon GWAS (N). Ces paramètres permettent au logiciel de recalibrer les statistiques d'association pour tenir compte de la polygénicité, comme décrit dans [Liang _et al. medRxiv_ 2024](https://www.biorxiv.org/content/10.1101/2023.10.17.562831v2). 

**Pourquoi cette correction est-elle importante ?** Les traits polygéniques présentent de nombreux variants causaux avec des tailles d'effet faibles dispersés à travers le génome. Plusieurs de ces variants seront en DL avec les eQTL inclus dans les modèles de prédiction d'expression génique, ce qui pourrait conduire à des associations fallacieuses. En fournissant ces statistiques, S-PrediXcan peut recalibrer les statistiques d'association TWAS pour atténuer cet effet de confusion.

In [ ]:
tissues="Adipose_Subcutaneous Artery_Coronary Brain_Cortex Liver Muscle_Skeletal Pancreas Whole_Blood"

for tissue in $tissues; do
    echo
    echo
    echo "Running S-PrediXcan for tissue ${tissue}"
    echo
    SPrediXcan.py \
        --model_db_path ../data/PrediXcan/predictdb/en_${tissue}.db \
        --covariance ../data/PrediXcan/predictdb/en_${tissue}.txt.gz \
        --gwas_file ../data/PrediXcan/gwas_harmonized/harmonized_en_${tissue}.tsv.gz \
        --snp_column rsid_x \
        --effect_allele_column effect_allele \
        --non_effect_allele_column other_allele \
        --output_file /workshop/local/results/twas_en_${tissue}.csv \
        --beta_column beta \
        --se_column standard_error \
        --gwas_h2 0.15 \
        --gwas_N 1000000
done

<div class="alert alert-info">
    <strong>Exploration des résultats</strong><br />
    Une fois l'exécution terminée, vous pouvez ouvrir un des fichiers de résultats dans le répertoire <code>results</code> pour examiner les résultats TWAS par vous-même. Chaque fichier contient les statistiques d'association entre les gènes prédits et le diabète de type 2 pour le tissu correspondant.
</div>

### Prochaines étapes

Vous avez maintenant généré vos premières analyses TWAS en utilisant S-PrediXcan. Dans le [prochain carnet](1b-S-PrediXcan-interpretation.ipynb), nous procéderons à l'interprétation détaillée de ces résultats TWAS en utilisant le langage R. Nous apprendrons à :

- Identifier les gènes significativement associés au DT2
- Visualiser les résultats à travers différents tissus
- Comprendre les implications biologiques de nos découvertes
- Appliquer les corrections pour tests multiples appropriées

Le carnet suivant vous permettra de découvrir quels gènes pourraient jouer un rôle dans le développement du diabète de type 2.